# Using Rubin OpSims in Simulations

A critical aspect to producing realistic simulations of time varying phenomena is correctly modeling the cadence at which the objects will be observed and capturing the relevant physical details about the observation. TDAstro provides an `OpSim` module that can load a Rubin opsim file and use that to filter observations based on location, associate observations with their filter, apply detector noise, etc.

This notebook provides an introduction to using opsim data in the simulations.

## OpSim Class

The `OpSim` object provides a wrapper for loading and querying survey-specific information including pointings and survey information. The required information is:
  * the pointing data (RA, dec, and times for each pointing), and
  * the zero point information for each pointing (a zero point column or the information needed to derive it)

Other common information includes the airmass, exposure time, and filter used (the information needed to derive the zero points).

Internally, the `OpSim` objects use a table with the column names are given by the imported data. For Rubin opsim files, this means columns will have names such as "observationStartMJD" or "fieldRA". Since different inputs may have different columns, the class provides the ability to map simple column names such as "ra" and "time" to the corresponding table names. The constructor allows the user to pass a column-mapping dictionary that maps the short column name to the name used within the database. The default column-mapper corresponds to the Rubin opsim format:
  * "airmass" -> "airmass"
  * "dec" -> "fieldDec"
  * "exptime" -> "visitExposureTime"
  * "filter" -> "filter"
  * "ra" -> "fieldRA"
  * "time" -> "observationStartMJD"
By default `OpSim` will look for Rubin column names, such as "observationStartMJD" and "fieldRA".

We can create a simple `OpSim` by manually specifying the data as a dict or a pandas dataframe.

In [ ]:
import numpy as np

from tdastro.opsim.opsim import OpSim

input_data = {
    "observationStartMJD": np.array([0.0, 1.0, 2.0, 3.0, 4.0]),
    "fieldRA": np.array([15.0, 30.0, 15.0, 0.0, 60.0]),
    "fieldDec": np.array([-10.0, -5.0, 0.0, 5.0, 10.0]),
    "zp_nJy": np.ones(5),
}
ops_data = OpSim(input_data)
print(ops_data.table)

Once we have an `OpSim` object, we can use the `[]` notation to access data directly. `OpSim` has the same mechanics as a pandas data frame. The columns are named and correspond to input attributes. Each row provides the information for a single pointing.

As noted above, the column mapping allows us to access column by either their given name ("observationStartMJD") or their shortened name ("time").

In [ ]:
ops_data["observationStartMJD"]  # Same as ops_data["time"]

## Loading an OpSim

Most users will want to load a pre-existing `OpSim` from a database file using the `from_db()` function or `from_url()` functions.  The `from_url()` function will download the opsim db from a given URL to TDAstro's data cache (if needed) and load the opsim from a file.  For Rubin, a large number of OpSims can be found at [https://s3df.slac.stanford.edu/data/rubin/sim-data/](https://s3df.slac.stanford.edu/data/rubin/sim-data/). For many studies you will want the most recent version of a baseline survey. But some users might want older versions or alternate approaches.

In [ ]:
opsim_url = "https://s3df.slac.stanford.edu/data/rubin/sim-data/sims_featureScheduler_runs3.4/baseline/baseline_v3.4_10yrs.db"

# Uncomment the following line to load data from a URL
# opsim_data = OpSim.from_url(opsim_url)

If the file has already been downloaded, the code will not redownload it unless the user sets `force_download=True`.

Alternatively if you are using a local OpSim with a known file path, you can use `from_db()` to read it directly.

In [ ]:
opsim_file = "../../tests/tdastro/data/opsim_shorten.db"
ops_data = OpSim.from_db(opsim_file)

print(f"Loaded an opsim database with {len(ops_data)} entries.")
print(f"Columns: {ops_data.columns}")

## Spatial Matching

`OpSim` provides a framework for efficiently determining when an object was observed given its (RA, Dec). We use the `range_search()` function to retrieve all pointings within a given radius of the query point.

We start by taking the (RA, Dec) of the first observation in the table and using that to determine all times this position was observed.

In [ ]:
query_ra = ops_data["ra"][0]
query_dec = ops_data["dec"][0]
print(f"Searching for ({query_ra}, {query_dec}).")

# Find everything within 0.5 degrees of the query point.
matches = ops_data.range_search(query_ra, query_dec, 0.5)
print(f"Found {len(matches)} matches at {matches}")

Once we have the indices, we can use those to find the other information about the pointings.

In [ ]:
ops_data["time"][matches]

We can run the spatial search in batch mode by providing a lists of RA and Dec. The `range_search()` will return a list of numpy arrays where each element in the top-level list represents the matches for a single query (RA, Dec). 

In [ ]:
num_queries = 10
query_ra = ops_data["ra"][0:num_queries]
query_dec = ops_data["dec"][0:num_queries]

matches = ops_data.range_search(query_ra, query_dec, 0.5)
for idx, m_ids in enumerate(matches):
    print(f"{idx}: ({query_ra[idx]}, {query_dec[idx]}) matched {m_ids}")

## Applying Noise

The `OpSim` object can use the information provided in its table to compute the noise properties for each observation and use those to inject synthetic noise into the observations. The `bandflux_error_point_source` function takes the pre-noise flux densities (bandflux of the point source in nJy) and returns the simulated bandflux noise in nJy.


In [ ]:
fluxes = np.array([100.0, 200.0, 300.0])
flux_err = ops_data.bandflux_error_point_source(fluxes, [0, 1, 2])
flux_err

This can then be feed into the `apply_noise` utility function to apply it to the flux.

In [ ]:
from tdastro.astro_utils.noise_model import apply_noise

noisy_fluxes = apply_noise(fluxes, flux_err)
noisy_fluxes

## Filtering

Depending on the analysis we run, we might want to use only a subset of the data.  `OpSim` provides a helper function `filter_rows()` that can be used to filter the data in an `OpSim` and thus speed up subsequent operations. This function creates a new copy of the data set with a subset of the rows.

In [ ]:
input_data["filter"] = np.array(["r", "g", "g", "r", "g"])
ops_data2 = OpSim(input_data)

print("BEFORE:")
print(ops_data2.table)

ops_data3 = ops_data2.filter_rows(ops_data2["filter"] == "g")

print("\n\nAFTER:")
print(ops_data3.table)

In addition to filtering the rows in the table, the function will also update the cached data structures within the `OpSim` object, such as the KD-tree.

## Scalability and Testing

To test the scalability of the OpSim we can user the `create_random_opsim()` helper function to generate completely random data by sampling uniformly over the surface of a sphere. This will not be a realistic survey since it will cover both the Northern and Southern hemisphere, but can provide a good test set for timing.

In [ ]:
from tdastro.opsim.opsim import create_random_opsim

# Create an opsim with 1,000,000 observations.
num_obs = 1_000_000
opsim_data2 = create_random_opsim(num_obs)

# Use the first 10,000 samples as queries.
num_queries = 100_000
query_ra = np.asarray(opsim_data2["ra"][0:num_queries])
query_dec = np.asarray(opsim_data2["dec"][0:num_queries])

Using the large OpSim data, we can test where batching speeds up the range search.

In [ ]:
%%timeit
for i in range(num_queries):
    _ = opsim_data2.range_search(query_ra[i], query_dec[i], 0.5)

In [ ]:
%%timeit
_ = opsim_data2.range_search(query_ra, query_dec, 0.5)

## Sampling

We can sample RA, Dec, and time from an `OpSim` object using the `OpSimRADECSampler` node. This will select a random observation from the OpSim and then create a random (RA, Dec) from near the center of that observation.

This allows us to generate fake query locations for testing that are compatible with arbitrary OpSims as opposed to generating a large number of fake locations and then filtering only those that match the opsim. The distribution will be weighted by the opsim's pointing. If it points at region A 90% of time time and region B 10% of the time, then 90% of the resulting points will be from region A and 10% from region B.

In [ ]:
from tdastro.math_nodes.ra_dec_sampler import OpSimRADECSampler

sampler_node = OpSimRADECSampler(ops_data, in_order=True)

# Test we can generate a single value.
(ra, dec, time) = sampler_node.generate(num_samples=10)

for i in range(10):
    print(f"{i}: ({ra[i]}, {dec[i]}) at t={time[i]}")